# **Acoustic Extinguisher Fire Dataset**

#### **TODO：后面记得修改注释、md文字的格式、大小等！**
This is the Coursework 2 for ***COMP4131 Data Modelling and Analysis***. The code follows Google Cloud's Data Science Road Map.

In [1]:
# Libraries used in this project
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import os
# import time
# from datetime import datetime
# from wordcloud import WordCloud, STOPWORDS
# from scipy import stats

## Stage 1 · Data Engineering

### Step 1 · Data Ingestion
【Step Description】
- The data is collected from [Acoustic-Extinguisher-Fire-Dataset - Kaggle](https://www.kaggle.com/datasets/muratkokludataset/acoustic-extinguisher-fire-dataset/data).
- I have downloaded the dataset from **Kaggle**, so the first step is reading it from an XLSX file.

In [2]:
# Read data from an XLSX file
df_raw = pd.read_excel("Acoustic_Extinguisher_Fire_Dataset.xlsx", "A_E_Fire_Dataset", index_col=None, na_values=["NA"])
df_raw

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
0,1,gasoline,10,96,0.0,75,0
1,1,gasoline,10,96,0.0,72,1
2,1,gasoline,10,96,2.6,70,1
3,1,gasoline,10,96,3.2,68,1
4,1,gasoline,10,109,4.5,67,1
...,...,...,...,...,...,...,...
17437,7,lpg,190,86,2.2,5,0
17438,7,lpg,190,84,2.0,4,0
17439,7,lpg,190,80,1.5,3,0
17440,7,lpg,190,76,0.4,2,0


### Step 2 · Data Preparation
【Step Description】
- Cleaning, deduplication, and integration of datasets. Handling missing values and ensuring data consistency.

##### **Task** 【Statistic summary and information】

In [3]:
# Check a quick statistic summary of dataframe
df_raw.describe(include='all')

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
count,17442.000000,17442,17442.000000,17442.000000,17442.000000,17442.000000,17442.000000
unique,NaN,4,NaN,NaN,NaN,NaN,NaN
top,NaN,gasoline,NaN,NaN,NaN,NaN,NaN
freq,NaN,5130,NaN,NaN,NaN,NaN,NaN
mean,3.411765,NaN,100.000000,96.379142,6.975634,31.611111,0.497821
std,1.750977,NaN,54.773826,8.164096,4.736169,20.939149,0.500010
min,1.000000,NaN,10.000000,72.000000,0.000000,1.000000,0.000000
25%,2.000000,NaN,50.000000,90.000000,3.200000,14.000000,0.000000
50%,3.000000,NaN,100.000000,95.000000,5.800000,27.500000,0.000000
75%,5.000000,NaN,150.000000,104.000000,11.200000,47.000000,1.000000


In [4]:
# Check info of dataframe
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17442 entries, 0 to 17441
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SIZE       17442 non-null  int64  
 1   FUEL       17442 non-null  object 
 2   DISTANCE   17442 non-null  int64  
 3   DESIBEL    17442 non-null  int64  
 4   AIRFLOW    17442 non-null  float64
 5   FREQUENCY  17442 non-null  int64  
 6   STATUS     17442 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 954.0+ KB


In [5]:
# Check the first ten rows
df_raw.head(10)

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
0,1,gasoline,10,96,0.0,75,0
1,1,gasoline,10,96,0.0,72,1
2,1,gasoline,10,96,2.6,70,1
3,1,gasoline,10,96,3.2,68,1
4,1,gasoline,10,109,4.5,67,1
5,1,gasoline,10,109,7.8,66,1
6,1,gasoline,10,103,9.7,65,1
7,1,gasoline,10,95,12.0,60,1
8,1,gasoline,10,102,13.3,55,1
9,1,gasoline,10,93,15.4,52,1


In [6]:
# Check the last five rows
df_raw.tail(10)

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
17432,7,lpg,190,91,1.6,10,0
17433,7,lpg,190,92,2.0,9,0
17434,7,lpg,190,91,1.9,8,0
17435,7,lpg,190,91,1.6,7,0
17436,7,lpg,190,87,2.5,6,0
17437,7,lpg,190,86,2.2,5,0
17438,7,lpg,190,84,2.0,4,0
17439,7,lpg,190,80,1.5,3,0
17440,7,lpg,190,76,0.4,2,0
17441,7,lpg,190,72,0.0,1,0


In [7]:
# Check data types of each column 
df_raw.dtypes

SIZE           int64
FUEL          object
DISTANCE       int64
DESIBEL        int64
AIRFLOW      float64
FREQUENCY      int64
STATUS         int64
dtype: object

In [8]:
# List all unique values of features and label
columns = ["SIZE", "FUEL", "DISTANCE", "DESIBEL", "AIRFLOW", "FREQUENCY", "STATUS"]
for column in columns:
    print(f"Column '{column}' having values:\n{np.sort(df_raw[column].unique())}\n")

Column 'SIZE' having values:
[1 2 3 4 5 6 7]

Column 'FUEL' having values:
['gasoline' 'kerosene' 'lpg' 'thinner']

Column 'DISTANCE' having values:
[ 10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170 180
 190]

Column 'DESIBEL' having values:
[ 72  74  75  76  78  79  80  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113]

Column 'AIRFLOW' having values:
[ 0.   0.4  0.8  1.   1.1  1.3  1.4  1.5  1.6  1.7  1.9  2.   2.1  2.2
  2.3  2.5  2.6  2.7  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.6  3.7  3.8
  3.9  4.   4.2  4.3  4.4  4.5  4.6  4.8  4.9  5.   5.2  5.3  5.4  5.6
  5.7  5.8  6.   6.1  6.3  6.4  6.5  6.7  6.8  7.   7.1  7.2  7.3  7.4
  7.5  7.7  7.8  7.9  8.1  8.2  8.3  8.5  8.6  8.7  8.8  8.9  9.   9.1
  9.2  9.3  9.5  9.6  9.7  9.9 10.  10.2 10.3 10.4 10.5 10.6 10.7 10.9
 11.  11.2 11.3 11.5 11.6 11.8 11.9 12.  12.2 12.3 12.5 12.6 12.8 12.9
 13.  13.1 13.2 13.3 13.4 13.5 13.6 13.8 13.9 14.

In [9]:
# Output all rows having NaN
df_raw[df_raw.isna().any(axis=1)]

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS


**Analysis according to the above statistic summary and information**:
1. The result shows that there are 17442 rows (data) and 7 columns (features) in the original dataset, which is consistent with the description in three referenced papers.
2. The result shows that there is no missing value in the original dataset. **=> Related to the missing value issue**.
3. The data type of features `FUEL` and `AIRFLOW` are `object` and `float64`, respectively. According to three original papers, they are categorical features, and other features are numerical. In subsequent operations, it is possible to convert features `FUEL` and `AIRFLOW` to `category` type for further analysis.
4. According to the statistic summary of the dataframe, the original dataset consists of six features (`SIZE`, `FUEL`, `DISTANCE`, `DESIBEL`, `AIRFLOW`, `FREQUENCY`) and one label field (`STATUS`). Checking the min, 25th percentiles, 50th percentiles, 75th percentiles and max value, there is no significant difference or gap among them. The minimum value and maximum value are consistent with those in the description of the original dataset. To be specific,
   
   |        | Value Range | Description | Unit | Data Type |
   | ------ | ------ | ------ | ------ | ------ |
   | `SIZE` | {1, 2, 3, 4, 5, 6, 7} | Fuel container size representative of flame size (1: 7cm for liquid fuels; 2: 12cm for liquid fuels; 3: 14cm for liquid fuels; 4: 16cm for liquid fuels; 5: 20cm for liquid fuels; 6: Half throttle setting for LPG; 7: Full throttle setting for LPG) | cm | numerical |
   | `FUEL` | {'gasoline', 'thinner', 'kerosene', 'lpg'} | Fuel type (first three values are liquid fuels) | / | categorical |
   | `DISTANCE` | [10, 190] | Distance (from flame to collimator output) | cm | numerical |
   | `DESIBEL` | [72, 113] | Sound pressure level | dB | numerical |
   | `AIRFLOW` | [0, 17] | Air flow generated by sound waves | m/s | numerical |
   | `FREQUENCY` | [1, 75] | Low frequency range | Hz | numerical |
   | `STATUS` | {0, 1} | Flame extinguished state (0: non-extinction state; 1: extinction state) | / | categorical |
    
5. The names of six features and one label are uppercase. In order to reduce complexity and avoid misinterpretation, I will rename them. **=> Related to the column renaming issue**.
6. I will check whether there are duplicates. **=> Related to duplicate issue**.
7. I will check whether there are outliers and bad data, especially for checking features like `AIRFLOW` and `FREQUENCY`. **=> Related to outliers and bad data issues**.

##### **Task** 【Dealing with duplicates】

In [10]:
# Show the duplicated items (rows or samples) in the original dataset
df_raw[df_raw.duplicated(keep=False)]

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS


**Analysis according to the above duplicate issue**:
1. There is no duplicated row (sample) in the original dataset.
2. To show my awareness of how to handle this issue, here is information about duplicate issues:
   - [Definition] It is possible that data entry errors/merging of datasets led to this issue.
   - [Methods] Identify and remove duplicates.

##### **Task** 【Dealing with outliers and bad data】
Using quantile() method to check.

In [11]:
# Check feature 'AIRFLOW'
# 1) Show the 5th percentile
df_raw['AIRFLOW'].quantile(0.05)

np.float64(0.0)

In [12]:
# 2) Show rows whose 'AIRFLOW' is lower than the 5th percentile
df_raw_5_airflow_quantile = df_raw[df_raw['AIRFLOW'] < df_raw['AIRFLOW'].quantile(0.05)]
df_raw_5_airflow_quantile

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS


In [13]:
# 3) Show the 95th percentile
df_raw['AIRFLOW'].quantile(0.95)

np.float64(14.9)

In [14]:
# 4) Show rows whose 'AIRFLOW' is greater than the 95th percentile
df_raw_95_airflow_quantile = df_raw[df_raw['AIRFLOW'] > df_raw['AIRFLOW'].quantile(0.95)]
df_raw_95_airflow_quantile

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
9,1,gasoline,10,93,15.4,52,1
10,1,gasoline,10,93,15.1,51,1
11,1,gasoline,10,95,15.2,50,1
12,1,gasoline,10,110,15.4,48,1
13,1,gasoline,10,111,15.2,47,1
...,...,...,...,...,...,...,...
16541,7,lpg,30,110,16.1,42,1
16542,7,lpg,30,110,15.2,40,1
16543,7,lpg,30,110,15.2,38,1
16594,7,lpg,40,107,15.2,44,1


In [15]:
# 5) Check unique values of feature 'AIRFLOW'
np.sort(df_raw_95_airflow_quantile['AIRFLOW'].unique())

array([15. , 15.1, 15.2, 15.4, 15.5, 15.7, 16. , 16.1, 16.3, 16.6, 17. ])

In [16]:
# Check feature 'FREQUENCY'
# 1) Show the 5th percentile
df_raw['FREQUENCY'].quantile(0.05)

np.float64(3.0)

In [17]:
# 2) Show rows whose 'FREQUENCY' is lower than the 5th percentile
df_raw_5_frequency_quantile = df_raw[df_raw['FREQUENCY'] < df_raw['FREQUENCY'].quantile(0.05)]
df_raw_5_frequency_quantile

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
52,1,gasoline,10,83,3.1,2,1
53,1,gasoline,10,75,0.8,1,1
106,1,gasoline,20,78,3.7,2,1
107,1,gasoline,20,76,1.0,1,0
160,1,gasoline,30,78,4.5,2,1
...,...,...,...,...,...,...,...
17333,7,lpg,170,76,0.0,1,0
17386,7,lpg,180,80,1.4,2,0
17387,7,lpg,180,74,0.0,1,0
17440,7,lpg,190,76,0.4,2,0


In [18]:
# 3) Check unique values of feature 'FREQUENCY'
np.sort(df_raw_5_frequency_quantile['FREQUENCY'].unique())

array([1, 2])

In [19]:
# 4) Show the 95th percentile
df_raw['FREQUENCY'].quantile(0.95)

np.float64(70.0)

In [20]:
# 5) Show rows whose 'FREQUENCY' is greater than the 95th percentile
df_raw_95_frequency_quantile = df_raw[df_raw['FREQUENCY'] > df_raw['FREQUENCY'].quantile(0.95)]
df_raw_95_frequency_quantile

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
0,1,gasoline,10,96,0.0,75,0
1,1,gasoline,10,96,0.0,72,1
54,1,gasoline,20,96,0.0,75,0
55,1,gasoline,20,96,0.0,72,0
108,1,gasoline,30,106,0.0,75,0
...,...,...,...,...,...,...,...
17281,7,lpg,170,97,0.0,72,0
17334,7,lpg,180,104,0.0,75,0
17335,7,lpg,180,97,0.0,72,0
17388,7,lpg,190,104,0.0,75,0


In [21]:
# 6) Check unique values of feature 'FREQUENCY'
np.sort(df_raw_95_frequency_quantile['FREQUENCY'].unique())

array([72, 75])

**Analysis according to the above outliers and bad data issues**:
1. After checking the *5th percentile* and *95th percentile*, I find that these data are not out of place. As a result, I keep these data unchanged.
2. For other features, the min, 25th percentiles, 50th percentiles, 75th percentiles and max value are normal according to the statistic summary of the dataframe shown before.
3. To show my awareness of how to handle this issue, here is information about outliers and bad data issues:
   - [Definition] There is a significant difference between some data points and other observations.
   - [Identification Methods]
     1. Z-score \
        Use standard deviation to measure how far away a data point is from the mean.
     2. IQR / Interquartile Range \
        Identify outliers as data points outside ```1.5 × IQR```. To be specific, ```IQR = Q3 - Q1```.
   - [Handling Methods]
     1. Remove rows having outliers.
     2. Replace outliers with thresholds (upper/lower).
     3. Apply log / square root transformations to reduce the impact of outliers.

##### **Task** 【Dealing with missing values】
According to the analysis of the above statistic summary and information, there is no missing value in the original dataset.
- To show my awareness of how to handle this issue, here is information about the missing value issue:
   - [Definition] Missing data may occur due to errors in data collection, storage or processing.
   - [Methods] `isnull()` or `isna()` method in Pandas.

##### **Task** 【Dealing with column renaming issue】

In [22]:
# create a mapping
mapping = {
    'SIZE': 'Size',
    'FUEL': 'Fuel',
    'DISTANCE': 'Distance',
    'DESIBEL': 'Desibel',
    'AIRFLOW': 'Airflow',
    'FREQUENCY': 'Frequency',
    'STATUS': 'Status'
}
# rename columns
df_raw = df_raw.rename(columns=mapping)
df_raw

,Size,Fuel,Distance,Desibel,Airflow,Frequency,Status
0,1,gasoline,10,96,0.0,75,0
1,1,gasoline,10,96,0.0,72,1
2,1,gasoline,10,96,2.6,70,1
3,1,gasoline,10,96,3.2,68,1
4,1,gasoline,10,109,4.5,67,1
...,...,...,...,...,...,...,...
17437,7,lpg,190,86,2.2,5,0
17438,7,lpg,190,84,2.0,4,0
17439,7,lpg,190,80,1.5,3,0
17440,7,lpg,190,76,0.4,2,0


**Analysis according to the above column renaming issue**:
1. Successfully renamed columns.

##### **Task** 【Check the dataframe again】

In [23]:
# Check a quick statistic summary of dataframe
df_raw.describe(include='all')

,Size,Fuel,Distance,Desibel,Airflow,Frequency,Status
count,17442.000000,17442,17442.000000,17442.000000,17442.000000,17442.000000,17442.000000
unique,NaN,4,NaN,NaN,NaN,NaN,NaN
top,NaN,gasoline,NaN,NaN,NaN,NaN,NaN
freq,NaN,5130,NaN,NaN,NaN,NaN,NaN
mean,3.411765,NaN,100.000000,96.379142,6.975634,31.611111,0.497821
std,1.750977,NaN,54.773826,8.164096,4.736169,20.939149,0.500010
min,1.000000,NaN,10.000000,72.000000,0.000000,1.000000,0.000000
25%,2.000000,NaN,50.000000,90.000000,3.200000,14.000000,0.000000
50%,3.000000,NaN,100.000000,95.000000,5.800000,27.500000,0.000000
75%,5.000000,NaN,150.000000,104.000000,11.200000,47.000000,1.000000


In [24]:
# Check info of dataframe
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17442 entries, 0 to 17441
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Size       17442 non-null  int64  
 1   Fuel       17442 non-null  object 
 2   Distance   17442 non-null  int64  
 3   Desibel    17442 non-null  int64  
 4   Airflow    17442 non-null  float64
 5   Frequency  17442 non-null  int64  
 6   Status     17442 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 954.0+ KB


In [25]:
# Output all rows having NaN
df_raw[df_raw.isna().any(axis=1)]

,Size,Fuel,Distance,Desibel,Airflow,Frequency,Status


##### **Task** 【Save in a file】

In [27]:
# Save in a file with setting the label of index
df_raw.to_csv('AEF_CLEAN.csv', index_label='Id')
# Copy the dataframe for subsequent operations
df_clean = df_raw.copy()

### Step 3 · Data Storage
【Step Description】
- Centralized storage using data lakes, warehouses, or cloud storage.
- Examples: AWS S3, Google BigQuery

【Note】
- [Information on the virtual environment]
  1. Name: `COMP4131gpu`
  2. Content: The virtual environment contains libraries used in this project.
  3. Management: Utilise **Anaconda3 Client** or **Anaconda Prompt** to install and uninstall relevant libraries.
  4. Version: `Anaconda Navigator 1.9.7`
- Because I handle, retrieve and save all files through the **Jupyter Notebook in the virtual environment of Anaconda3**, there is no need to use data lakes, warehouses or cloud storage. **是否实现待定，去看看google bigquery这个工具**

### Step 4 · Data Cataloging
【Step Description】
- Metadata management for better discoverability. **这里的实现待定** \
【**这一行供参考，后面删除这行文字**】Tools: Data catalog services to enable seamless data discovery

## Stage 2 · Data Analysis

### Step 1 · Data Exploration
【Step Description】
- Perform exploratory data analysis (EDA) using visualizations and summary statistics.

### Step 2 · Data Preprocessing (Feature Engineering)
【Step Description】
- Transform raw data into meaningful features for modeling. \
【**这一行供参考，后面删除这行文字**】Techniques: normalization, encoding categorical variables, creating derived features

### Step 3 · Data Insights
【Step Description】
- Generate actionable insights to understand trends and patterns. \
【**这一行供参考，后面删除这行文字**】Identifying correlation, seasonality, or data anomalies.

## Stage 3 · Model Development

### Step 1 · Feature Engineering
【Step Description】
- Improve model performance by selecting, transforming, and creating features.

### Step 2 · Model Training
【Step Description】
- Train machine learning models using algorithms such as linear regression, decision trees, neural networks, etc. \
【**这一行供参考，后面删除这行文字**】Tools: Scikit-learn, TensorFlow, PyTorch.

### Step 3 · Model Evaluation
【Step Description】
- Assess the model’s performance using metrics (e.g., accuracy, precision, recall, F1-score). \
Perform cross-validation to validate the model’s robustness.

### Step 4 · Model Registry
【Step Description】
- Version control and management of models to ensure traceability.

## Stage 4 · Machine Learning Engineering

### Step 1 · Model Development
【Step Description】
- Deploy models as APIs, microservices, or edge devices. **这里的实现待定（大概率不实现），若不实现需要说明** \
- Deployment types: Batch, real-time, and on-device. **这里的实现待定（大概率不实现），若不实现需要说明**

### Step 2 · Model Serving
【Step Description】
- Serve predictions to applications or dashboards. **这里的实现待定（大概率不实现），若不实现需要说明** \
- Infrastructure tools: Kubernetes, Docker, AWS Lambda. **这里的实现待定（大概率不实现），若不实现需要说明**

### Step 3 · Model Monitoring
【Step Description】
- Track model performance in production (accuracy drift, latency). **这里的实现待定（大概率不实现），若不实现需要说明** \
- Monitor data for changes that can affect model performance. **这里的实现待定（大概率不实现），若不实现需要说明**

## Stage 5 · Insights Activation

【Step Description】
1. Influence Business Decisions:
  - Use insights to generate reports, dashboards, and alerts.
  - Example: Dashboards showing real-time KPIs for decision-making.
**这里的实现待定（大概率不实现），若不实现需要说明**

2. Influence Consumer Decisions:
  - Personalize recommendations, offers, and experiences.
  - Example: Reducing customer churn or increasing user engagement.
**这里的实现待定（大概率不实现），若不实现需要说明**

3. Serve Other Applications/Services:
  - Integrate insights with external systems (e.g., customer service platforms, fraud detection).
**这里的实现待定（大概率不实现），若不实现需要说明**

> **Paper References**:
> 1. Paper 1: \
>    [Koklu, M., & Taspinar, Y. S. (2021). Determining the extinguishing status of fuel flames with sound wave by machine learning methods. IEEE access, 9, 86207-86216, Doi: 10.1109/ACCESS.2021.3088612](https://ieeexplore.ieee.org/document/9452168)
> 2. Paper 2: \
>    [Taspinar, Y. S., Koklu, M., & Altin, M. (2021). Classification of flame extinction based on acoustic oscillations using artificial intelligence methods. Case Studies in Thermal Engineering, 28, 101561.](https://www.sciencedirect.com/science/article/pii/S2214157X21007243)
> 3. Paper 3: \
>    [Taspinar, Y. S., Koklu, M., & Altin, M. (2022). Acoustic-driven airflow flame extinguishing system design and analysis of capabilities of low frequency in different fuels. Fire technology, 58(3), 1579-1597.](https://link.springer.com/article/10.1007/s10694-021-01208-9)
> 
> **Other References**:
> 1. [Read data from excel](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html#pandas.read_excel)
> 2. [Comments in Python](https://docs.python.org/3/reference/lexical_analysis.html)
> 3. [Comments in Python](https://blog.csdn.net/weixin_69553582/article/details/136881191)
> 4. [Official website of Pandas](https://pandas.pydata.org/docs/index.html)
> 5. 

In [ ]:
## read data from two CSV files
df_raw_dreaddit_train = pd.read_csv("dreaddit-train.csv")
df_raw_dreaddit_test = pd.read_csv("dreaddit-test.csv")

In [ ]:
## check basic information about these two datasets
# 1. generate descriptive statistics
df_raw_dreaddit_train.describe(include='all')

In [ ]:
df_raw_dreaddit_train.info(verbose=True)

In [ ]:
df_raw_dreaddit_test.describe(include='all')

In [ ]:
df_raw_dreaddit_test.info(verbose=True)

**References:**
- [isin()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html#pandas.DataFrame.isin)

In [ ]:
# 2. check first five rows
df_raw_dreaddit_train.head()

In [ ]:
df_raw_dreaddit_test.head()

In [ ]:
## select the rows (post data) posted in the subreddit "anxiety" or "stress"
# 1. for the original training dataset from 'dreaddit'
df_train = df_raw_dreaddit_train.loc[df_raw_dreaddit_train['subreddit'].isin(['anxiety', 'stress']), ['id', 'subreddit', 'post_id', 'text', 'label', 'confidence', 'social_timestamp', 'sentiment']]
df_train

In [ ]:
# 2. for the original testing dataset from 'dreaddit'
df_test = df_raw_dreaddit_test.loc[df_raw_dreaddit_test['subreddit'].isin(['anxiety', 'stress']), ['id', 'subreddit', 'post_id', 'text', 'label', 'confidence', 'social_timestamp', 'sentiment']]
df_test

In [ ]:
## select the rows (post data) posted not in the subreddit "anxiety" or "stress", aiming at selecting posts containing and related to 'depression' to fill the datasets
# 1. for the original training dataset from 'dreaddit'
df_remaining_train = df_raw_dreaddit_train.loc[~df_raw_dreaddit_train['subreddit'].isin(['anxiety', 'stress']), ['id', 'subreddit', 'post_id', 'text', 'label', 'confidence', 'social_timestamp', 'sentiment']]
df_remaining_train

In [ ]:
# 2. for the original testing dataset from 'dreaddit'
df_remaining_test = df_raw_dreaddit_test.loc[~df_raw_dreaddit_test['subreddit'].isin(['anxiety', 'stress']), ['id', 'subreddit', 'post_id', 'text', 'label', 'confidence', 'social_timestamp', 'sentiment']]
df_remaining_test

In [ ]:
## merge these two remaining datasets
df_remaining = pd.concat([df_remaining_train, df_remaining_test], axis=0).reset_index(drop=True) # axis=0: concat two tables vertically
df_remaining

- [contains()](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html#pandas.Series.str.contains)

In [ ]:
## select posts whose contents contain several keywords from the remaining dataset 
## keywords include: "depression", "depressed", "hopeless", "hopelessness", "sadness", "sad", "helpless", "helplessness", "disappoint", "disappointed", "disappointment"
keywords = ["depression", "depressed", "hopeless", "hopelessness", "sadness", "sad", "helpless", "helplessness", "disappoint", "disappointed", "disappointment"]
df_remaining_depression = df_remaining[df_remaining['text'].str.contains('|'.join(keywords))]
df_remaining_depression

In [ ]:
## merge three datasets into one dataset for data preprocessing
# 1. merge 'df_train' and 'df_test' into one dataframe
df_1 = pd.concat([df_train, df_test], axis=0).reset_index(drop=True) # axis=0: concat two tables vertically
df_1

In [ ]:
# 2. merge 'df_1' and 'df_remaining_depression' into one dataframe
dataset_1 = pd.concat([df_1, df_remaining_depression], axis=0).reset_index(drop=True) # axis=0: concat two tables vertically
dataset_1

In [ ]:
# 3. rename the 'social_timestamp'
mapping = {
    'id': 'id',
    'subreddit': 'subreddit',
    'post_id': 'post_id',
    'text': 'text',
    'label': 'label',
    'confidence': 'confidence',
    'social_timestamp': 'created_utc',
    'sentiment': 'sentiment'
}
dataset_1 = dataset_1.rename(columns=mapping)
dataset_1

In [ ]:
# 4. check basic information about the 'dataset_1'
dataset_1.describe(include='all')

In [ ]:
# 4. save the final dataset as 'dataset_1'
dataset_1.to_csv('dataset-1.csv', index=False)

In [ ]:
## read data from the CSV file
df_raw_twitter = pd.read_csv("twitter.csv")

In [ ]:
## check basic information about the dataset
# 1. generate descriptive statistics
df_raw_twitter.describe(include='all')

In [ ]:
# 2. check first five rows
df_raw_twitter.head()

In [ ]:
## get the raw dataset
# 1. select specific columns (features)
df_twitter = df_raw_twitter[['id', 'conversation_id', 'tweet', 'created_at', 'date']]
df_twitter

In [ ]:
# 2. add a column 'label' with all rows value 1 according to the description of the raw dataset
df_twitter['label'] = 1
df_twitter

**Reference**:
- [rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html#pandas.DataFrame.rename)

In [ ]:
# 3. rename all features
mapping = {
    'id': 'id',
    'conversation_id': 'post_id',
    'tweet': 'text',
    'created_at': 'created_utc',
    'date': 'date', 
    'label': 'label'
}
df_twitter = df_twitter.rename(columns=mapping)
df_twitter

In [ ]:
# 4. rearrange the order of features
df_twitter = df_twitter[['id', 'post_id', 'text', 'label', 'created_utc', 'date']]
df_twitter

In [ ]:
## select posts whose contents contain several keywords from the remaining dataset 
## keywords include: "depression", "depressed", "hopeless", "hopelessness", "sadness", "sad", "helpless", "helplessness", "disappoint", "disappointed", "disappointment"
keywords = ["depression", "depressed", "hopeless", "hopelessness", "sadness", "sad", "helpless", "helplessness", "disappoint", "disappointed", "disappointment"]
df_twitter_depression = df_twitter[df_twitter['text'].str.contains('|'.join(keywords), na=False)] # using na=True to handle 'NaN'
df_twitter_depression

**References**:
- [Merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge)

In [ ]:
## merge this dataset with dataset_1 for data preprocessing
dataset_2 = pd.concat([dataset_1, df_twitter_depression], axis=0).reset_index(drop=True) # axis=0: concat two tables vertically
dataset_2

In [ ]:
## rearrange the order of features
main_dataset = dataset_2[['id', 'subreddit', 'post_id', 'text', 'label', 'confidence', 'created_utc', 'date', 'sentiment']]
main_dataset

In [ ]:
## convert the date into datetime type
main_dataset['date'] = pd.to_datetime(main_dataset['date'])

In [ ]:
## save the final main dataset into a CSV file
main_dataset.to_csv('main-dataset.csv', index=False)

In [ ]:
main_dataset.info()

In [ ]:
## Configuration for PRAW
reddit = praw.Reddit(
    client_id="gOxImWE64i1kd4qjZtsUGA",
    client_secret="dQ0PfogVu7r75MOSo1tOeYMjBk8y_A",
    password="", # my account has no password, so this field is empty
    user_agent="python:depression:v0.0.1 (by /u/Efficient_Draw_2434)",
    username="u/Efficient_Draw_2434",
)

**References:**
- [submission](https://praw.readthedocs.io/en/stable/code_overview/models/submission.html)
- [comments](https://praw.readthedocs.io/en/stable/code_overview/models/comment.html)
- methods of subreddit
  1. hot
  2. new
  3. top
  4. rising
  5. controversial
  6. gilded

In [ ]:
## Get 100 submissions and their related comments and replies about the topic of 'depression'
# 1. initialisation
subreddit = reddit.subreddit("depression") # subreddit about depression
subs = [] # store submissions related to 'depression'
coms = [] # store comments related to 'depression'
reps = [] # store replies related to 'depression'
num_sub = 0 # collected number of submissions
num_com = 0 # collected number of comments
num_rep = 0 # collected number of replies
start_time = time.perf_counter() # record consuming time

# 2. get submissions
flag_comment = False
flag_reply = False
hot_list = subreddit.hot(limit=100) # method of subreddit: hot
if hot_list:
    print("There exist submissions.")
    for submission in hot_list:
        # select unremoved non-stickied submissions that are not related to the rules of this subreddit
        if not submission.stickied and submission.selftext not in ["[removed]", "[deleted]"] and submission:
            num_sub += 1
            # contruct a dict to store the data
            submission_info = {
                'id': submission.id,
                'subreddit': 'depression',
                'post_id': submission.id,
                # 'name': submission.name, # fullname of the submission
                # 'title': submission.title, # title of the submission
                'text': submission.selftext, # content of the submission
                # 'num_comments': submission.num_comments, # number of comments in this submission
                # 'score': submission.score, # the number of upvotes for the submission
                'created_utc': submission.created_utc, # created time
                # 'upvote_ratio': submission.upvote_ratio, # the percentage of upvotes from all votes on the submission
                # 'author_id': None, # info about the author of this comment
                # 'author_username': None, # info about the author of this comment
                # 'author_descriptions': None # info about the author of this comment
            }
            # # get info about the author of this submission
            # if submission.author:
            #     if submission.author.name not in ["[removed]", "[deleted]"]:
            #         # if not submission.author.is_suspended:
            #         #     submission_info['author_id'] = submission.author.id
            #         submission_info['author_username'] = submission.author.name
            #         # check whether this author has subreddits
            #         # if submission.author.subreddit and hasattr(submission.author, "subreddit"):
            #         #     submission_info['author_descriptions'] = submission.author.subreddit.public_description
            # append the submission
            subs.append(submission_info)
        
            # 3. get comments
            submission.comments.replace_more(limit=None) # get all comments
            com_list = submission.comments
            if com_list:
                flag_comment = True
                for comment in com_list:
                    # select unremoved comments
                    if comment and comment.body and comment.body not in ["[removed]", "[deleted]"]:
                        num_com += 1
                        # contruct a dict to store the data
                        comment_info = {
                            'id': comment.id,
                            'subreddit': 'depression',
                            'post_id': comment.id,
                            # 'submission_id': None, # id of the submission this comment belongs to
                            # 'submission_title': None, # title of the submission this comment belongs to
                            'text': comment.body, # content of the comment
                            # 'link_id': comment.link_id, # the submission ID that the comment belongs to
                            # 'score': comment.score, # the number of upvotes for the comment
                            'created_utc': comment.created_utc, # created time
                            # 'author_id': None, # info about the author of this comment
                            # 'author_username': None, # info about the author of this comment
                            # 'author_descriptions': None # info about the author of this comment
                        }
                        # # get info about the submission this comment belongs to
                        # if comment.submission:
                        #     # comment_info['submission_id'] = comment.submission.id
                        #     comment_info['submission_title'] = comment.submission.title
                        # # get info about the author of this comment
                        # if comment.author:
                        #     if comment.author.name not in ["[removed]", "[deleted]"]:
                        #         # if not comment.author.is_suspended:
                        #         #     comment_info['author_id'] = comment.author.id
                        #         comment_info['author_username'] = comment.author.name
                        #         # check whether this author has subreddits
                        #         # if comment.author.subreddit and hasattr(comment.author, "subreddit"):
                        #         #     comment_info['author_descriptions'] = comment.author.subreddit.public_description
                        # append the comment
                        coms.append(comment_info)
        
                        # 4. get replies
                        comment.replies.replace_more(limit=None) # get all replies
                        rep_list = comment.replies
                        if rep_list:
                            flag_reply = True
                            for reply in rep_list:
                                # select unremoved replies
                                if reply and reply.body and reply.body not in ["[removed]", "[deleted]"]:
                                    num_rep += 1
                                    # contruct a dict to store the data
                                    reply_info = {
                                        'id': reply.id,
                                        'subreddit': 'depression',
                                        'post_id': reply.id,
                                        # 'submission_id': None, # id of the submission this reply belongs to
                                        # 'submission_title': None, # title of the submission this reply belongs to
                                        'text': reply.body, # content of the comment
                                        # 'link_id': reply.link_id, # the submission ID that the comment belongs to
                                        # 'score': reply.score, # the number of upvotes for the comment
                                        'created_utc': reply.created_utc, # created time
                                        # 'author_id': None, # info about the author of this reply
                                        # 'author_username': None, # info about the author of this reply
                                        # 'author_descriptions': None # info about the author of this reply
                                    }
                                    # # get info about the submission this reply belongs to
                                    # if reply.submission:
                                    #     # reply_info['submission_id'] = reply.submission.id
                                    #     reply_info['submission_title'] = reply.submission.title
                                    # # get info about the author of this reply
                                    # if reply.author:
                                    #     if reply.author.name not in ["[removed]", "[deleted]"]:
                                    #         # if not reply.author.is_suspended:
                                    #         #     reply_info['author_id'] = reply.author.id
                                    #         reply_info['author_username'] = reply.author.name
                                    #         # check whether this author has subreddits
                                    #         # if reply.author.subreddit and hasattr(reply.author, "subreddit"):
                                    #         #     reply_info['author_descriptions'] = reply.author.subreddit.public_description
                                    # append the reply
                                    reps.append(reply_info)
else:
    print("No submission exists.")
if not flag_comment:
    print("No comment exists.")
if not flag_reply:
    print("No reply exists.")

# 5. output-related information
end_time = time.perf_counter() # record consuming time
print("The data collection procedure consumes: ", end_time - start_time, " seconds.")
print("\nThe list of submissions: \n", subs)
print("\nThe list of comments: \n", coms)
print("\nThe list of replies: \n", reps)
print("\nNumber of submissions: ", num_sub)
print("Number of comments: ", num_com)
print("Number of replies: ", num_rep)

In [ ]:
## convert submissions, comments and replies into 'dataframe'
# 1. submissions
df_subs = pd.DataFrame(subs)
df_subs

In [ ]:
# show the first five rows of df_subs
df_subs.head()

In [ ]:
# 2. comments
df_coms = pd.DataFrame(coms)
df_coms

In [ ]:
# show the first five rows of df_coms
df_coms.head()

In [ ]:
# 3. replies
df_reps = pd.DataFrame(reps)
df_reps

In [ ]:
# show the first five rows of df_reps
df_reps.head()

In [ ]:
## merge into one dataframe
# 1. concat dataframe of comments and replies
df_coms_reps = pd.concat([df_coms, df_reps], axis=0).reset_index(drop=True) # axis=0: concat two tables vertically
# 2. merge it and dataframe of submission
test_dataset = pd.concat([df_subs, df_coms_reps], axis=0).reset_index(drop=True) # axis=0: concat two tables vertically
test_dataset

In [ ]:
## store data as a CSV file
test_dataset.to_csv('test-dataset.csv', index=False)

In [ ]:
## explore the dataset to find potential issues
# 1. print the shape
print(test_dataset.shape)
test_dataset

In [ ]:
# 2. Show the description of all columns
test_dataset.describe(include='all')

In [ ]:
# 3. check whether there exist any missing value
test_dataset[test_dataset.isna().any(axis=1)]

In [ ]:
# 4. Print the info of the main dataset
test_dataset.info()

The result of the description of the `DataFrame` shows:
- **Because the data is collected from the API, it is possible to have different texts with different frequencies.** The value with the highest frequency of column 'text' is 'Count me in' with a frequency of 2.
- **Missing Value Issue**: The test dataset lacks three columns: 'label', 'confidence' and 'sentiment'.

In [ ]:
## Dealing with duplicates
# 1. show the duplicated items (rows or samples) in the original dataset
test_dataset[test_dataset.duplicated(keep=False)]

- **Because the data is collected from the API, it is possible to have different texts with different frequencies. The code here is used for 'Count me in' example.** \
  The value with the highest frequency of column 'text' is 'Count me in' with a frequency of 2.

In [ ]:
# 2. For the above phenomenon, check whether they are posted by one person
test_dataset[test_dataset['text'] == 'Count me in']

If there are duplicated texts 'Count me in', it doesn't mean that there exists a problem. In reality, it is possible.

- **Missing Value Issue**: The test dataset lacks column 'confidence'. \
  Because the origin dataset from Reddit doesn't have any value in this column, it is a good idea to replace the missing values with the average of the subgroup whose data was collected from Kaggle and the subreddit is 'anxiety'.

In [ ]:
## Create a new column 'confidence'
# 1. get the average of the subgroup in the main dataset
mean_value = main_dataset.loc[main_dataset['subreddit'] == 'depression', 'confidence'].values[0]
mean_value

In [ ]:
# 2. iterate over all rows
test_dataset['confidence'] = 0.0
for i, row in test_dataset[:].iterrows():
   test_dataset.loc[i, 'confidence'] = mean_value
test_dataset

In [ ]:
# 3. show rows having 'NaN'
test_dataset[test_dataset['confidence'].isna()]

- **Missing Value Issue**: The test dataset lacks column 'label'.

In [ ]:
## Create a new column 'label'
# 1. imputation
test_dataset['label'] = -1
test_dataset

In [ ]:
# 2. show rows having 'NaN'
test_dataset[test_dataset['label'].isna()]

- **Missing Value Issue**: The test dataset lacks column 'sentiment'.

In [ ]:
## Create a new column 'label'
# 1. imputation
test_dataset['sentiment'] = 0.0
test_dataset

In [ ]:
# 2. show rows having 'NaN'
test_dataset[test_dataset['sentiment'].isna()]

- **Creating a new column 'date'**. \
  To keep consistent with the field structure of `main_dataset`, it is necessary to add the 'date' column, whose value is converted from the 'created_utc' column.

In [ ]:
## Create a column named 'date'
# 1. check info
test_dataset.info()

In [ ]:
# 2. show rows having 'NaN'
test_dataset[test_dataset['created_utc'].isna()]

In [ ]:
# 3. iterate over all rows
for i, row in test_dataset[:].iterrows():
    # 1) get current row's UTC timestamp
    timestamp = test_dataset.loc[i, 'created_utc']
    # print(f"orginal timestamp: {timestamp}")
    # 2) convert to date
    obj = datetime.utcfromtimestamp(timestamp)
    # 3) formatted as yyyy-MM-dd HH:mm:ss
    date = obj.strftime('%Y-%m-%d %H:%M:%S')
    # print(f"formatted date: {date}\n")
    # 4) save the formatted date
    test_dataset.loc[i, 'date'] = date
# conver the date into datetime type
test_dataset['date'] = pd.to_datetime(test_dataset['date'])
test_dataset

In [ ]:
# 4. rearrange the order of features
test_dataset = test_dataset[['id', 'subreddit', 'post_id', 'text', 'label', 'confidence', 'created_utc', 'date', 'sentiment']]
test_dataset

**Final Check and Save Data**.

In [ ]:
## check whether there exist any missing value
test_dataset[test_dataset.isna().any(axis=1)]

In [ ]:
## save the preprocessed test dataset
test_dataset.to_csv('preprocessed-test-dataset.csv', index=False)

In [ ]:
## explore the dataset to find potential issues
# 1. print the shape
print(global_dataset.shape)
global_dataset

In [ ]:
# 2. show the description of all columns
global_dataset.describe(include='all')

In [ ]:
# 3. check whether there are missing values and other issues
print(global_dataset[global_dataset.isna().any(axis=1)].shape)
global_dataset[global_dataset.isna().any(axis=1)]

In [ ]:
# 4. check whether other columns () have missing values
print(global_dataset[global_dataset[['country/region', 'year', 'schizophrenia_disorders_rate', 
                                     'bipolar_disorders_rate', 'eating_disorders_rate', 'anxiety_disorders_rate',
                                     'drug_disorders_rate', 'depression_disorders_rate', 
                                     'alcohol_disorders_rate']].isna().any(axis=1)].shape)
global_dataset[global_dataset[['country/region', 'year', 'schizophrenia_disorders_rate',
                               'bipolar_disorders_rate', 'eating_disorders_rate', 'anxiety_disorders_rate',
                               'drug_disorders_rate', 'depression_disorders_rate', 
                               'alcohol_disorders_rate']].isna().any(axis=1)]

The result of the description of the `DataFrame` shows:
- **Missing Value Issue**: The result shows that 980 rows are missing the unicode for the country/region. **However, this feature is not the key feature of the task, we can just replace them with 'unknown'.**

In [ ]:
## Dealing with duplicates
# 1. show the duplicated items (rows or samples) in the original dataset
global_dataset[global_dataset.duplicated(keep=False)]

The result shows that there is no duplicated row.

In [ ]:
## Dealing with missing values
# 1. show rows having 'NaN'
global_dataset[global_dataset['unicode'].isna()]

In [ ]:
# 2. iterate over the rows where 'unicode' is 'NaN' and replace these unicode with 'unknown'
for i, row in global_dataset[global_dataset['unicode'].isna()].iterrows():
    global_dataset.loc[i, 'unicode'] = 'unknown'
global_dataset

In [ ]:
# 3. check again
global_dataset[global_dataset['unicode'].isna()]

Successfully!

In [ ]:
## Dealing with outliers and bad data
# According to the description of all columns, there is no unreasonable data.

**Final Check and Save Data**.

In [ ]:
## check whether there exist any missing value
global_dataset[global_dataset.isna().any(axis=1)]

In [ ]:
## save the preprocessed main dataset
global_dataset.to_csv('preprocessed-global-dataset.csv', index=False)

In [ ]:
## Check info of the main dataset and test dataset
# 1. main dataset
main_dataset

In [ ]:
# 2. test dataset
test_dataset

In [ ]:
# 3. merge two datasets
final_social_dataset = pd.concat([main_dataset, test_dataset], axis=0).reset_index(drop=True) # axis=0: concat two tables vertically
final_social_dataset

In [ ]:
# 4. check info 
final_social_dataset.info()

In [ ]:
# 5. check whether there exist any missing value
final_social_dataset[final_social_dataset.isna().any(axis=1)]

In [ ]:
## Copy the main dataset
nltk_main_dataset = final_social_dataset.copy()
nltk_main_dataset

In [ ]:
## Data Preprocessing
# 1. get English stop words
stop_words = stopwords.words('english')
print(f"English stop words include: \n{stop_words}")
# 2. create an instance of TweetTokenizer
tknzr = TweetTokenizer()

In [ ]:
# 3. iterate over all rows
# 1) create a variable for counting overall work frequency distribution
overall_tokens = []
for i, row in nltk_main_dataset[:].iterrows():
    print(f"index: {i} \n1) id: {nltk_main_dataset.loc[i, 'id']} \n2) text: \n{row['text']}")
    # 2) lowercase
    lower_text = row['text'].lower()
    # print(f"3) lower: \n{lower_text}")
    # 3) remove HTML tags
    soup = BeautifulSoup(lower_text, 'html.parser')
    removed_html_text = soup.get_text()
    # print(f"4) remove html texts: \n{removed_html_text}")
    # 4) remove punctuation
    #    str.maketrans: create a character Map
    #      1st param: characters to be replaced
    #      2nd param: characters after replacement
    #      3rd param: characters to be deleted
    # removed_punctuation_text = removed_html_text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    removed_punctuation_text = removed_html_text.replace('’', '\'') # this punctuation has a really high frequency
    # print(f"5) remove punctuation: \n{removed_punctuation_text}")
    # 5) tokenize
    # tokens = nltk.word_tokenize(removed_html_text, language='english')
    tokens = tknzr.tokenize(removed_punctuation_text)
    # print(f"6) tokens: \n{tokens}")
    # 6) remove stop words
    non_stopword_tokens = [token for token in tokens if token not in stop_words]
    print(f"3) tokens without stop words: \n{non_stopword_tokens}\n")
    # 7) append items from iterable to the end of the array
    overall_tokens.extend(non_stopword_tokens)
# 8) show preprocessed dataframe
nltk_main_dataset

In [ ]:
## Construct a self-defined depression dictionary based on high-frequency words
# 1. get the word frequency distribution (without considering punctuation)
overall_tokens = [token for token in overall_tokens if token not in string.punctuation]
fdist = FreqDist(token for token in overall_tokens)
print(f"word frequency distribution of all tokens: \n{fdist.pformat(maxlen=200)}\n")

According to the above frequency dictionary, **I chose tokens with a frequency larger than 90 to construct a self-defined depression dictionary based on high-frequency words.**

In [ ]:
# 2. select depression-related words manually
extra_depression_list = [
    "anxiety", "life", "want", "can't", "never", "depression",  
    "need", "bad", "sad", "nothing", "hard", "depressed", "anxious", 
    "mental", "panic", "worse"
]
# extra_depression_list = [
#     "anxiety", "help", "life", "can't", "depression", "never", 
#     "sad", "bad", "anxious", "need", "mental", "hard", "health", 
#     "nothing", "depressed", "attacks", "issues"
# ]
extra_depression_list

In [ ]:
# 3. compute a strength score for each depression-related word selected above
extra_depression_dict = {}
for word in extra_depression_list:
    # 1) create the analyzer
    sia = SentimentIntensityAnalyzer()
    # 2) get the score
    score = sia.lexicon.get(word, 0) # if the word is not in the sia's dictionary, then return 0
    print(f"The sentiment score for '{word}' is: {score}")
    # 3) construct a key-value pair
    if score != 0: # in the sia's dictionary
        extra_depression_dict[word] = score
    else: # set a default strength score
        extra_depression_dict[word] = -0.1
extra_depression_dict

In [ ]:
## VADER sentiment analysis
# 1. iterate over all rows and call the method
nltk_main_dataset['pred_label'] = np.nan
nltk_main_dataset['pred_sentiment'] = np.nan
for i, row in nltk_main_dataset[:].iterrows():
    print(f"index: {i} \n1) id: {nltk_main_dataset.loc[i, 'id']} \n2) subreddit: {nltk_main_dataset.loc[i, 'subreddit']} \n3) text: \n{row['text']}")
    # 1) create the analyzer
    sia = SentimentIntensityAnalyzer()
    # 2) extend the dictionary of the analyzer
    sia.lexicon.update(extra_depression_dict)
    # 3) compute the sentiment scores for sentiment strength based on the input text
    sentiment_scores = sia.polarity_scores(row['text'])
    # 4) output details of the score
    sorted_sentiment_scores = sorted(sentiment_scores)
    for j in sorted_sentiment_scores:
        print('4) {0}: {1}; '.format(j, sentiment_scores[j]), end='')
    print("\n")
    # 5) classify by confidence score
    if sentiment_scores['compound'] < 0:
        nltk_main_dataset.loc[i, 'pred_label'] = 1 # create a new column
    else:
        nltk_main_dataset.loc[i, 'pred_label'] = 0 # create a new column
    # 6) store the confidence scores and label
    nltk_main_dataset.loc[i, 'pred_sentiment'] = sentiment_scores['compound'] # create a new column
nltk_main_dataset

In [ ]:
# 2. compare the sentiment scores with the real sentiment scores (including data from all subreddits, except for the 'depression' subreddit)
eval_main_dataset = nltk_main_dataset[nltk_main_dataset['subreddit'] != 'depression'].copy()
pd.DataFrame({'real_sentiment': eval_main_dataset['sentiment'], 'pred_sentiment': eval_main_dataset['pred_sentiment']})

In [ ]:
# 3. compute the accuracy of the result
cnt_sentiment = ((eval_main_dataset['sentiment'] > 0) == (eval_main_dataset['pred_sentiment'] > 0)).sum()
acc_sentiment = cnt_sentiment / len(eval_main_dataset)
print(f"The consistent amount of data is: {cnt_sentiment} and overall amount of data is: {len(eval_main_dataset)}.")
print(f"The accuracy of the result is: {acc_sentiment}.")

In [ ]:
# 4. compare the labels with the real labels (including data from all subreddits, except for the 'depression' subreddit)
pd.DataFrame({'real_label': eval_main_dataset['label'], 'pred_label': eval_main_dataset['pred_label']})

In [ ]:
# 5. compute the accuracy of the result
cnt_label = (eval_main_dataset['label'] == eval_main_dataset['pred_label']).sum()
acc_label = cnt_label / len(eval_main_dataset)
print(f"The consistent amount of data is: {cnt_label} and overall amount of data is: {len(eval_main_dataset)}.")
print(f"The accuracy of the result is: {acc_label}.")

The result shows that for data collected from the first dataset (whose data is collected form the Reddit), the NLTK can output the accuracy of almost 69.01%.

In [ ]:
## Result analysis
# 1. show the word cloud
# 1) read the mask image
mask_picture = np.array(Image.open(os.path.join(os.getcwd(), "people.png"))) # os.getcwd(): get current directory
# 2) create an instance of the WordCloud
wc = WordCloud(background_color="white", max_words=2000, mask=mask_picture, 
               width=800, height=600, stopwords=stop_words, 
               contour_width=1, contour_color='steelblue')
# 3) concat all tokens into a string
wc_text = ' '.join(overall_tokens)
# 4) generate the word cloud
wc.generate(wc_text)
# 5) save to a file
wc.to_file(os.path.join(os.getcwd(), "word_cloud.png"))
# 6) show the word cloud
plt.imshow(wc, interpolation='bilinear')
plt.title("Word Cloud of keywords")
plt.axis("off")
plt.figure()
plt.show()

In [ ]:
# 2. create a heatmap
# 1) use keywords that appear in the Data Collection Part
depression_keywords = ["depression", "depressed", "hopeless", "hopelessness", "sadness", "sad", "helpless", "helplessness", "disappoint", "disappointed", "disappointment"]
# 2) copy the main_dataset
analyse_main_dataset = nltk_main_dataset.copy()
analyse_main_dataset

In [ ]:
# 3) check info
analyse_main_dataset.info()
analyse_main_dataset['pred_label'] = analyse_main_dataset['pred_label'].astype(int)
# 4) extract year, month and day
analyse_main_dataset['year'] = analyse_main_dataset['date'].dt.year
analyse_main_dataset['month'] = analyse_main_dataset['date'].dt.month
analyse_main_dataset['day'] = analyse_main_dataset['date'].dt.day
analyse_main_dataset

In [ ]:
analyse_main_dataset.info()

In [ ]:
# 5) construct a dataframe to draw the heatmap
# select rows mentioning depression-related keywords
#   case=False: case insensitive
#   na=False: not fill missing values
mention_main_dataset = analyse_main_dataset[analyse_main_dataset['text'].str.contains('|'.join(depression_keywords), case=False, na=False)]
mention_main_dataset

- [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.mean.html#pandas.core.groupby.DataFrameGroupBy.mean)
- [reset_index()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html#pandas.DataFrame.reset_index)
- [pivot()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html#pandas.DataFrame.pivot)
- [fillna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna)
- [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html#seaborn.heatmap)

In [ ]:
# use groupby() to group by year and month
heatmap_data = mention_main_dataset.groupby(['year', 'month']).size()
heatmap_data = heatmap_data.reset_index(name='num_keywords')
heatmap_data

In [ ]:
# sort the dataframe
sorted_heatmap_data = heatmap_data.sort_values(by=['year', 'month'], ascending=True)
sorted_heatmap_data

In [ ]:
# reshape the dataframe
sorted_heatmap_data = sorted_heatmap_data.pivot(index='year', columns='month', values='num_keywords')
sorted_heatmap_data

In [ ]:
# replace all NaN with 0
sorted_heatmap_data = sorted_heatmap_data.fillna(0)
sorted_heatmap_data

In [ ]:
# draw the heatmap
plt.figure(figsize=(8, 6))
ax = sns.heatmap(data=sorted_heatmap_data, annot=True, fmt=".1f", linewidth=.5, cbar=True, cmap="crest")
ax.set(xlabel="month", ylabel="year")
ax.xaxis.tick_top()
# set title and save
plt.title("Heatmap of depression mentioned by year and month")
plt.savefig('heatmap.png', dpi=300)

In [ ]:
# 3. correlation analysis
# 1) use groupby() to group by year
num_keywords_data = mention_main_dataset.groupby(['year']).size()
num_keywords_data = num_keywords_data.reset_index(name='num_keywords')
num_keywords_data

In [ ]:
# 2) compute the global average annual prevalence of depression from the global mental health dataset
average_prevalence_depression = global_dataset.groupby('year')[['schizophrenia_disorders_rate', 'bipolar_disorders_rate', 
                                                                'eating_disorders_rate', 'anxiety_disorders_rate', 
                                                                'drug_disorders_rate', 'depression_disorders_rate', 
                                                                'alcohol_disorders_rate']].mean()
average_prevalence_depression = average_prevalence_depression.reset_index(names="year")
average_prevalence_depression

In [ ]:
# 3) integrate two dataframes into one dataframe
regression_data = pd.merge(num_keywords_data, average_prevalence_depression, on='year', how='outer')
regression_data

In [ ]:
# 4) consider rows without missing values
# here, I don't reset indices, because it can help find the corresponding year
handled_regression_data = regression_data.dropna()
handled_regression_data = handled_regression_data[['num_keywords', 'schizophrenia_disorders_rate', 'bipolar_disorders_rate',
                                   'eating_disorders_rate', 'anxiety_disorders_rate', 'drug_disorders_rate', 
                                   'depression_disorders_rate', 'alcohol_disorders_rate']]
handled_regression_data

In [ ]:
# 5) calculate the correlation
correlation = handled_regression_data.corr(numeric_only=True)
correlation

**Analysis**:
- Each variable and itself have a perfect positive linear correlation.
- Variables `num_keywords` and `depression_disorders_rate` have a correlation coefficient of -0.356597. This means that **there is a moderate negative correlation** between these two variables. **As the number of keywords mentioned in the social media platforms increases, the rate of depression will decrease.**
- Variables `num_keywords` and `anxiety_disorders_rate` have a correlation coefficient of -0.739638. Variables `num_keywords` and `alcohol_disorders_rate` have a correlation coefficient of -0.761096. These mean that **there are strong negative correlations** between these pairs of two variables.
- Variables `num_keywords` and `schizophrenia_disorders_rate` have a correlation coefficient of 0.687847. Variables `num_keywords` and `bipolar_disorders_rate	` have a correlation coefficient of 0.703226. Variables `num_keywords` and `eating_disorders_rate` have a correlation coefficient of 0.675158. Variables `num_keywords` and `drug_disorders_rate` have a correlation coefficient of 0.693156. These mean that **there are strong positive correlations** between these pairs of two variables, respectively.

In [ ]:
# 6) create the heatmap for correlation
plt.figure(figsize=(10, 8))
sns.heatmap(data=correlation, annot=True, fmt=".2f", linewidth=.5, cbar=True, vmin=-1.0, vmax=1.0)
plt.title("Correlation between number of keywords and disorders rates")
plt.subplots_adjust(top=0.95, bottom=0.3, left=0.25, right=0.95, hspace=0.1, wspace=0.1)
plt.savefig('correlation.png', dpi=300)

In [ ]:
# 7) calculate the pearson correlation and p-value
pearson_coef, p_value = stats.pearsonr(handled_regression_data['num_keywords'], 
                                       handled_regression_data['depression_disorders_rate'])
print(f"The Pearson Correlation Coefficient is: {pearson_coef} with a p-value of P = {p_value}")

In [ ]:
# 4. draw a line chart
# 1) select data
line_data = regression_data[['year', 'num_keywords', 'depression_disorders_rate']].copy()
line_data

In [ ]:
# 2) imputate the missing values
line_data = line_data.fillna(0)
line_data

- [plt](https://matplotlib.org/stable/api/pyplot_summary.html)
- [plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot)
- [legend](https://matplotlib.org/stable/api/legend_api.html#module-matplotlib.legend)

In [ ]:
# 3) draw the chart (left axis)
plt.figure(figsize=(10, 6))
line1 = plt.plot(line_data['year'], line_data['num_keywords'], 'g^-', label='Number of keywords [having data between 2013 and 2025]') # return a list
plt.xlabel('Year')
plt.ylabel('Average number of keywords')
# 4) draw the chart (right axis)
ax2 = plt.twinx() # share the first axis
line2 = ax2.plot(line_data['year'], line_data['depression_disorders_rate'], 'ro-', label='Depression disorders rate (%) [having data between 1990 and 2017]') # return a list
ax2.set_ylabel('Average depression disorders rate (%)')
# 5) other settings of the line chart
plt.title("Average number of keywords and depression disorders rate throughout years")
lines = [line1[0], line2[0]]
labels = [line.get_label() for line in lines]
plt.legend(lines, labels, loc='center left', frameon=True)
# 6) save the line chart
plt.savefig('line_chart.png', dpi=300)

In [ ]:
## save the data
nltk_main_dataset.to_csv('nltk-main-dataset.csv', index=False)